In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.api as sm
import shap
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from  sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [93]:
import os
working_directory = os.getcwd()
print(working_directory)

/Users/ilinamitra


In [94]:
path = working_directory + '/Documents/df_updated.csv'
df = pd.read_csv(path, encoding='latin-1')
df.head()

,County,County.Code,suicides,Population,Crude_Rate,Uninsured: %,Mental_Health_Providers,Social_Associations,vet_pop_2010,Opioid Dispensing Rate per 100,...,in_labor_force,civilian_labot_force,employed,unemployed,armed_force,not_labor_force,total_occupied_housing,owner_occupied,renter_occupied,median_gross_rent
0,"Baldwin County, AL",1003,51,229287,22.242866,10.9,228,226,19614,65.0,...,99495,99317,95091,4226,178,72802,80930,60896,20034,1020
1,"Jefferson County, AL",1073,83,655342,12.665143,10.0,1379,953,42503,139.0,...,326672,326101,304994,21107,571,198562,261231,163799,97432,900
2,"Lee County, AL",1081,21,166831,12.587589,10.5,178,144,11532,35.4,...,79457,78849,74770,4079,608,50796,60237,37111,23126,856
3,"Madison County, AL",1089,54,379453,14.231011,9.9,613,403,33657,87.3,...,187581,186248,176910,9338,1333,104491,148189,99046,49143,857
4,"Mobile County, AL",1097,55,412716,13.326355,12.0,441,463,30347,98.0,...,186097,185327,174519,10808,770,141286,156251,100853,55398,853


In [95]:
df = df.drop(["Population","County", "County.Code","Year", "suicides","StateAbbr", "StateDesc", "LocationName", 
              "State", "...1"],axis=1)

In [72]:
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.8
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
#print(to_drop)

# Drop features with correlation greater than 0.8
df.drop(to_drop, axis=1, inplace=True)

In [73]:
df.shape

(447, 27)

In [74]:
df.columns

Index(['Crude_Rate', 'Uninsured: %', 'Mental_Health_Providers',
       'Opioid Dispensing Rate per 100', 'homicide_rate', 'RUCC_2013',
       'avg_temp', 'precip', 'Obesity among adults aged >=18 years',
       'Stroke among adults aged >=18 years',
       'Current asthma among adults aged >=18 years',
       'Depression among adults aged >=18 years',
       'Arthritis among adults aged >=18 years',
       'Binge drinking among adults aged >=18 years',
       'Current smoking among adults aged >=18 years',
       'Mental health not good for >=14 days among adults aged >=18 years',
       'Cholesterol screening among adults aged >=18 years',
       'Visits to doctor for routine checkup within the past year among adults aged >=18 years',
       'High cholesterol among adults aged >=18 years who have been screened in the past 5 years',
       'dem_perc', 'Score', 'Gun_Death_Rate_100k', 'black_pop', 'native_pop',
       'islander_pop', 'median_household_income', 'armed_force'],
      dtype

In [75]:
#KNN imputer instead? what would the neighbor for an NJ county be?
#does it make sense to just drop New Jersey from the analysis?

imputer = SimpleImputer(strategy='mean', missing_values=np.nan)

imputer = imputer.fit(df[['homicide_rate']])
df['homicide_rate'] = imputer.transform(df[['homicide_rate']])

imputer = imputer.fit(df[['Obesity among adults aged >=18 years']])
df['Obesity among adults aged >=18 years'] = imputer.transform(
    df[['Obesity among adults aged >=18 years']])

imputer = imputer.fit(df[['Stroke among adults aged >=18 years']])
df['Stroke among adults aged >=18 years'] = imputer.transform(df[['Stroke among adults aged >=18 years']])

imputer = imputer.fit(df[['Current asthma among adults aged >=18 years']])
df['Current asthma among adults aged >=18 years'] = imputer.transform(
    df[['Current asthma among adults aged >=18 years']])

imputer = imputer.fit(df[['Depression among adults aged >=18 years']])
df['Depression among adults aged >=18 years'] = imputer.transform(
    df[['Depression among adults aged >=18 years']])

imputer = imputer.fit(df[['Arthritis among adults aged >=18 years']])
df['Arthritis among adults aged >=18 years'] = imputer.transform(
    df[['Arthritis among adults aged >=18 years']])

imputer = imputer.fit(df[['Binge drinking among adults aged >=18 years']])
df['Binge drinking among adults aged >=18 years'] = imputer.transform(
    df[['Binge drinking among adults aged >=18 years']])

imputer = imputer.fit(df[['Current smoking among adults aged >=18 years']])
df['Current smoking among adults aged >=18 years'] = imputer.transform(
    df[['Current smoking among adults aged >=18 years']])

imputer = imputer.fit(df[['Mental health not good for >=14 days among adults aged >=18 years']])
df['Mental health not good for >=14 days among adults aged >=18 years'] = imputer.transform(
    df[['Mental health not good for >=14 days among adults aged >=18 years']])

imputer = imputer.fit(df[['Cholesterol screening among adults aged >=18 years']])
df['Cholesterol screening among adults aged >=18 years'] = imputer.transform(
    df[['Cholesterol screening among adults aged >=18 years']])

imputer = imputer.fit(df[['Visits to doctor for routine checkup within the past year among adults aged >=18 years']])
df['Visits to doctor for routine checkup within the past year among adults aged >=18 years'] = imputer.transform(
    df[['Visits to doctor for routine checkup within the past year among adults aged >=18 years']])

imputer = imputer.fit(df[['High cholesterol among adults aged >=18 years who have been screened in the past 5 years']])
df['High cholesterol among adults aged >=18 years who have been screened in the past 5 years'] = imputer.transform(
    df[['High cholesterol among adults aged >=18 years who have been screened in the past 5 years']])

In [76]:
df.isnull().sum()

Crude_Rate                                                                                  0
Uninsured: %                                                                                0
Mental_Health_Providers                                                                     0
Opioid Dispensing Rate per 100                                                              0
homicide_rate                                                                               0
RUCC_2013                                                                                   0
avg_temp                                                                                    0
precip                                                                                      0
Obesity among adults aged >=18 years                                                        0
Stroke among adults aged >=18 years                                                         0
Current asthma among adults aged >=18 years                 

In [77]:
y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (70/30)
    The R2 is maximized in cross-validation.
    '''
    
    random_state = 57
    
    # split the data
    #train on less? try 0.7 and then more in test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)
    #print('mean_xtrain': np.mean()) #df.mean() 
        
    #kfold split
    kf = KFold(n_splits=10,shuffle=True,random_state=24)
        
    # loop through the hyperparameter combinations or use GridSearchCV 
    grid = GridSearchCV(estimator = ML_algo, param_grid=param_grid,scoring = 'neg_mean_absolute_error', 
                            n_jobs = -1, cv = kf, return_train_score = True, verbose=True) 
        
    #fitting on training data
    grid.fit(X_train, y_train)
    results = pd.DataFrame(grid.cv_results_)

    print('best model parameters:',grid.best_params_)
    print('validation score:',grid.best_score_) # this is the mean validation score over all iterations
    
    # calculate and save the test score
    y_test_pred = grid.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print(test_score)
        
    return grid, test_score

ML_algo = Lasso()
param_grid = {
              'alpha':[0.0000000001, 0.00000000129, 0.0000000167, 0.000000215, 0.00000278, 0.0000359,
                      0.000464, 0.00599, 0.0774, 1],
             'max_iter': [100000]}

model, score = MLpipe(X,y,ML_algo,param_grid)
print(score) 

#look at errors of specific counties?

Fitting 10 folds for each of 10 candidates, totalling 100 fits
best model parameters: {'alpha': 0.00599, 'max_iter': 100000}
validation score: -2.6120385011169494
2.5674113461283734
2.5674113461283734


In [78]:
#having chosen the values from the param grid, running the model on 70% of data (training) and testing on 30% of data (test set)

y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (70/30)
    The R2 is maximized in cross-validation.
    '''
        
    random_state = 57

    # split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)


    model = ML_algo.fit(X_train, y_train)

    # calculate and save the test score
    y_test_pred = model.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print('test score:',test_score)
        
    return model, test_score

ML_algo = Lasso(alpha = 0.0774, max_iter = 100000)

model, score = MLpipe(X,y,ML_algo,param_grid)
print(score) 

test score: 2.5328299401401133
2.5328299401401133


In [79]:
#the sign of the coefficient usually stays the same but when the random state changes, the magnitude is subject to change
model.coef_

array([ 0.65569354, -0.1383649 ,  0.53385171, -0.19268558,  0.10537253,
       -0.        , -0.36094241, -1.32125977,  0.        ,  0.85209778,
        0.03798507,  2.31390793,  0.12252591,  0.        , -0.84975754,
       -0.04362256, -1.72493994,  0.02108843, -1.17866722, -0.29816188,
        1.15509011, -0.24643464,  0.24701261, -0.        , -0.69727388,
        0.60509995])

In [80]:
pd.Series(model.coef_, index = X.columns)

Uninsured: %                                                                                0.655694
Mental_Health_Providers                                                                    -0.138365
Opioid Dispensing Rate per 100                                                              0.533852
homicide_rate                                                                              -0.192686
RUCC_2013                                                                                   0.105373
avg_temp                                                                                   -0.000000
precip                                                                                     -0.360942
Obesity among adults aged >=18 years                                                       -1.321260
Stroke among adults aged >=18 years                                                         0.000000
Current asthma among adults aged >=18 years                                                

In [81]:
#run a regular regression on the test set and then gives statistical significance - stats model should have it in python
#fit, do CV, do variable selection and then refit a regular regression on the training and predict on the test set
#do a random forest - robustness check of the lasso results: gridsearchCV or randomsearchCV: won't give signs 

In [82]:
#Now drop stroke, smoking, islander pop, and average temp from the variable list and run again and run a 
#normal regression without the two variables mentioned above

In [83]:
df = df.drop(["avg_temp", "Current smoking among adults aged >=18 years",
             "Stroke among adults aged >=18 years", "islander_pop"],axis=1)

In [84]:
y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (70/30)
    The R2 is maximized in cross-validation.
    '''
        
    random_state = 57

    # split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)

    #Baseline linear regression model
    linreg = LinearRegression(fit_intercept=True)
    model = lingreg.fit(X_train, y_train)

    # calculate and save the test score
    y_test_pred = model.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print('test score:',test_score)
        
    return model, test_score
    
print(score)

2.5328299401401133


In [85]:
model.coef_

array([ 0.65569354, -0.1383649 ,  0.53385171, -0.19268558,  0.10537253,
       -0.        , -0.36094241, -1.32125977,  0.        ,  0.85209778,
        0.03798507,  2.31390793,  0.12252591,  0.        , -0.84975754,
       -0.04362256, -1.72493994,  0.02108843, -1.17866722, -0.29816188,
        1.15509011, -0.24643464,  0.24701261, -0.        , -0.69727388,
        0.60509995])

In [88]:
#pd.Series(model.coef_, index = X.columns)

In [87]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:             Crude_Rate   R-squared (uncentered):                   0.951
Model:                            OLS   Adj. R-squared (uncentered):              0.949
Method:                 Least Squares   F-statistic:                              376.2
Date:                Wed, 11 Jan 2023   Prob (F-statistic):                   9.62e-263
Time:                        21:35:44   Log-Likelihood:                         -1172.0
No. Observations:                 447   AIC:                                      2388.
Df Residuals:                     425   BIC:                                      2478.
Df Model:                          22                                                  
Covariance Type:            nonrobust                                                  
                                                                                               coef    std err          

In [ ]:
#Calculating VIF

In [173]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [174]:
path = working_directory + '/Documents/df_updated.csv'
df = pd.read_csv(path, encoding='latin-1')
df.head()

df = df.drop(["Population","County", "County.Code","Year", "suicides","StateAbbr", "StateDesc", "LocationName", 
              "State", "...1"],axis=1)

y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

In [175]:
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)

imputer = imputer.fit(df[['homicide_rate']])
df['homicide_rate'] = imputer.transform(df[['homicide_rate']])

imputer = imputer.fit(df[['firearm_death_rate']])
df['firearm_death_rate'] = imputer.transform(df[['firearm_death_rate']])

imputer = imputer.fit(df[['Stroke among adults aged >=18 years']])
df['Stroke among adults aged >=18 years'] = imputer.transform(df[['Stroke among adults aged >=18 years']])

imputer = imputer.fit(df[['Current lack of health insurance among adults aged 18-64 years']])
df['Current lack of health insurance among adults aged 18-64 years'] = imputer.transform(
    df[['Current lack of health insurance among adults aged 18-64 years']])

imputer = imputer.fit(df[['Obesity among adults aged >=18 years']])
df['Obesity among adults aged >=18 years'] = imputer.transform(
    df[['Obesity among adults aged >=18 years']])

imputer = imputer.fit(df[['Current asthma among adults aged >=18 years']])
df['Current asthma among adults aged >=18 years'] = imputer.transform(df[['Current asthma among adults aged >=18 years']])

imputer = imputer.fit(df[['Diagnosed diabetes among adults aged >=18 years']])
df['Diagnosed diabetes among adults aged >=18 years'] = imputer.transform(
    df[['Diagnosed diabetes among adults aged >=18 years']])

imputer = imputer.fit(df[['Depression among adults aged >=18 years']])
df['Depression among adults aged >=18 years'] = imputer.transform(
    df[['Depression among adults aged >=18 years']])

imputer = imputer.fit(df[['Arthritis among adults aged >=18 years']])
df['Arthritis among adults aged >=18 years'] = imputer.transform(
    df[['Arthritis among adults aged >=18 years']])

imputer = imputer.fit(df[['Chronic obstructive pulmonary disease among adults aged >=18 years']])
df['Chronic obstructive pulmonary disease among adults aged >=18 years'] = imputer.transform(
    df[['Chronic obstructive pulmonary disease among adults aged >=18 years']])

imputer = imputer.fit(df[['Binge drinking among adults aged >=18 years']])
df['Binge drinking among adults aged >=18 years'] = imputer.transform(
    df[['Binge drinking among adults aged >=18 years']])

imputer = imputer.fit(df[['Current smoking among adults aged >=18 years']])
df['Current smoking among adults aged >=18 years'] = imputer.transform(
    df[['Current smoking among adults aged >=18 years']])

imputer = imputer.fit(df[['Physical health not good for >=14 days among adults aged >=18 years']])
df['Physical health not good for >=14 days among adults aged >=18 years'] = imputer.transform(
    df[['Physical health not good for >=14 days among adults aged >=18 years']])

imputer = imputer.fit(df[['Coronary heart disease among adults aged >=18 years']])
df['Coronary heart disease among adults aged >=18 years'] = imputer.transform(
    df[['Coronary heart disease among adults aged >=18 years']])

imputer = imputer.fit(df[['No leisure-time physical activity among adults aged >=18 years']])
df['No leisure-time physical activity among adults aged >=18 years'] = imputer.transform(
    df[['No leisure-time physical activity among adults aged >=18 years']])

imputer = imputer.fit(df[['High blood pressure among adults aged >=18 years']])
df['High blood pressure among adults aged >=18 years'] = imputer.transform(
    df[['High blood pressure among adults aged >=18 years']])

imputer = imputer.fit(df[['Mental health not good for >=14 days among adults aged >=18 years']])
df['Mental health not good for >=14 days among adults aged >=18 years'] = imputer.transform(
    df[['Mental health not good for >=14 days among adults aged >=18 years']])

imputer = imputer.fit(df[['Mental health not good for >=14 days among adults aged >=18 years']])
df['Mental health not good for >=14 days among adults aged >=18 years'] = imputer.transform(
    df[['Mental health not good for >=14 days among adults aged >=18 years']])

imputer = imputer.fit(df[['Fair or poor self-rated health status among adults aged >=18 years']])
df['Fair or poor self-rated health status among adults aged >=18 years'] = imputer.transform(
    df[['Fair or poor self-rated health status among adults aged >=18 years']])

imputer = imputer.fit(df[['Cholesterol screening among adults aged >=18 years']])
df['Cholesterol screening among adults aged >=18 years'] = imputer.transform(
    df[['Cholesterol screening among adults aged >=18 years']])

imputer = imputer.fit(df[['Chronic kidney disease among adults aged >=18 years']])
df['Chronic kidney disease among adults aged >=18 years'] = imputer.transform(
    df[['Chronic kidney disease among adults aged >=18 years']])

imputer = imputer.fit(df[['Visits to doctor for routine checkup within the past year among adults aged >=18 years']])
df['Visits to doctor for routine checkup within the past year among adults aged >=18 years'] = imputer.transform(
    df[['Visits to doctor for routine checkup within the past year among adults aged >=18 years']])

imputer = imputer.fit(df[['Cancer (excluding skin cancer) among adults aged >=18 years']])
df['Cancer (excluding skin cancer) among adults aged >=18 years'] = imputer.transform(
    df[['Cancer (excluding skin cancer) among adults aged >=18 years']])

imputer = imputer.fit(df[['Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure']])
df['Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure'] = imputer.transform(
    df[['Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure']])

imputer = imputer.fit(df[['High cholesterol among adults aged >=18 years who have been screened in the past 5 years']])
df['High cholesterol among adults aged >=18 years who have been screened in the past 5 years'] = imputer.transform(
    df[['High cholesterol among adults aged >=18 years who have been screened in the past 5 years']])


In [178]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
#vif_data["VIF"] = [variance_inflation_factor(X.values, i)›
                          #for i in range(len(X.columns))]

vif_data["VIF"] = [variance_inflation_factor(df.loc[:, df.columns != 'Crude_Rate'].values, i) 
                   for i in range(len(df.loc[:, df.columns != 'Crude_Rate'].columns))]
vif_data

divide by zero encountered in double_scalars


,feature,VIF
0,Uninsured: %,7.797923e+01
1,Mental_Health_Providers,4.243989e+01
2,Social_Associations,8.244251e+01
3,vet_pop_2010,1.545846e+02
4,Opioid Dispensing Rate per 100,1.555214e+01
5,homicide_rate,1.860357e+02
6,firearm_death_rate,2.075422e+02
7,RUCC_2013,1.416475e+01
8,avg_temp,3.605651e+02
9,precip,2.260706e+01


In [163]:
#from population, keep native_pop based on negative correlations seen above, keep total_public assistance, unemployed,
#income_per_capita

In [179]:
df = df.drop(["total_population", "black_pop", "asian_pop", "white_pop",
         "islander_pop", "other_pop", "mixed_pop", "family_household", "married_couple_family", "other_family", 
              "single_man_family", 
         "single_woman_family", "nonfamily_household", "single_family_alone", "single_family_notalone", "no_school",
         "nursery", "kindergarten", "grade_1", "grade_2", "grade_3", "grade_4", "grade_5", "grade_6", "grade_7",
         "grade_8", "grade_9", "grade_10", "grade_11", "grade_12_no_diploma", "hs_diploma", "GED", "some_college",
         "some_college_no_degree", "associates_degree", "bachelors_degree", "professional_degree", "doctoral_degree",
        "under_0.5_ip", "0.5_0.9_ip", "1_1.24_ip", "1.25_1.49_ip", "1.5_1.84_ip", "1.85_1.99_ip", "over_2_ip",
        "assistance_income", "no_assistance_income", "total_emp", "in_labor_force", 
         "civilian_labot_force", "employed", "armed_force", "not_labor_force", "owner_occupied", "renter_occupied",
             "total_occupied_housing"],axis=1)

In [180]:
y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature

vif_data["VIF"] = [variance_inflation_factor(df.loc[:, df.columns != 'Crude_Rate'].values, i) 
                   for i in range(len(df.loc[:, df.columns != 'Crude_Rate'].columns))]

vif_data

,feature,VIF
0,Uninsured: %,60.291145
1,Mental_Health_Providers,12.756061
2,Social_Associations,47.050638
3,vet_pop_2010,14.758389
4,Opioid Dispensing Rate per 100,13.910328
5,homicide_rate,178.970576
6,firearm_death_rate,200.462424
7,RUCC_2013,12.136400
8,avg_temp,269.958043
9,precip,17.422974


In [181]:
df = df[['Crude_Rate', 'Mental_Health_Providers','vet_pop_2010', 'Opioid Dispensing Rate per 100',
        'RUCC_2013', 'precip', 'native_pop']]

In [182]:
y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature

vif_data["VIF"] = [variance_inflation_factor(df.loc[:, df.columns != 'Crude_Rate'].values, i) 
                   for i in range(len(df.loc[:, df.columns != 'Crude_Rate'].columns))]

vif_data

,feature,VIF
0,Mental_Health_Providers,3.053910
1,vet_pop_2010,5.582497
2,Opioid Dispensing Rate per 100,6.087741
3,RUCC_2013,4.301726
4,precip,5.142091
5,native_pop,2.958321


In [187]:
y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (70/30)
    The R2 is maximized in cross-validation.
    '''
    
    random_state = 7
    
    # split the data
    #train on less? try 0.7 and then more in test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)
    #print('mean_xtrain': np.mean()) #df.mean() 
        
    #kfold split
    kf = KFold(n_splits=10,shuffle=True,random_state=24)
        
    # loop through the hyperparameter combinations or use GridSearchCV 
    grid = GridSearchCV(estimator = ML_algo, param_grid=param_grid,scoring = 'neg_mean_absolute_error', 
                            n_jobs = -1, cv = kf, return_train_score = True, verbose=True) 
        
    #fitting on training data
    grid.fit(X_train, y_train)
    results = pd.DataFrame(grid.cv_results_)

    print('best model parameters:',grid.best_params_)
    print('validation score:',grid.best_score_) # this is the mean validation score over all iterations
    
    # calculate and save the test score
    y_test_pred = grid.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print(test_score)
        
    return grid, test_score

ML_algo = Lasso()
param_grid = {#'alpha':[0.01, 0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09, 0.10],
              'alpha':[0.0000000001, 0.00000000129, 0.0000000167, 0.000000215, 0.00000278, 0.0000359,
                      0.000464, 0.00599, 0.0774, 1],
             'max_iter': [100000]}

model, score = MLpipe(X,y,ML_algo,param_grid)
print(score) 

Fitting 10 folds for each of 10 candidates, totalling 100 fits
best model parameters: {'alpha': 0.0774, 'max_iter': 100000}
validation score: -3.4750342266108234
3.321930827991583
3.321930827991583


In [188]:
#having chosen the values from the param grid, running the model on 80% of data (training) and testing on 20% of data (test set)

y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (70/30)
    The R2 is maximized in cross-validation.
    '''
        
    random_state = 7

    # split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)


    model = ML_algo.fit(X_train, y_train)

    # calculate and save the test score
    y_test_pred = model.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print('test score:',test_score)
        
    return model, test_score

ML_algo = Lasso(alpha = 0.0774, max_iter = 100000)

model, score = MLpipe(X,y,ML_algo,param_grid)
print(score) 

test score: 3.321930827991583
3.321930827991583


In [189]:
model.coef_

array([-1.24729053, -0.        ,  1.12751405,  0.84600006, -0.28236145,
        0.93454369])

In [186]:
pd.Series(model.coef_, index = X.columns)

Mental_Health_Providers          -1.784151
vet_pop_2010                      0.367327
Opioid Dispensing Rate per 100    1.224177
RUCC_2013                         1.068491
precip                           -0.437209
native_pop                        1.176645
dtype: float64

In [ ]:
#does make them a little more stable but at what cost?